In [3]:
# ---- IMPORT SECTION ---- #
import gensim, logging
import os
from os import listdir
import os
from os.path import isfile, join
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models.keyedvectors import KeyedVectors
from array import array
import numpy as np
import nltk
import sys
from nltk.stem import WordNetLemmatizer
import treetaggerwrapper
from gensim.test.utils import datapath
import fasttext

In [2]:
#Mini = 1 rapport succint avec comment éxecuter le script etc...
#pouvoir guider le prof sur la correction
#On envoie que le code et il regenere tout

def create_cooc_mat(filePath,window):
    file = open(filePath,'r')
    matrice = {}
    with open(filePath)as file:
        data = file.read()
        i = 0
        for word in data.split():
            if word not in matrice:
                matrice[word] = {}
            for y in range(0,window):
                if i+y+1 < len(data.split()) and data.split()[i+y+1] not in matrice[word]:
                    matrice[word][data.split()[i+y+1]] = 1
                elif i+y+1 < len(data.split()) :
                    matrice[word][data.split()[i+y+1]] += 1
            i +=1
    return matrice
                
cooc_mat = create_cooc_mat('./English/-10-000-gold.txt',3)

{'<HEADLINE>': {'$10,000': 1, 'Gold?': 1, '<AUTHOR': 1}, '$10,000': {'Gold?': 1, '<AUTHOR': 1, 'name="Kenneth': 1}, 'Gold?': {'<AUTHOR': 1, 'name="Kenneth': 1, 'Rogoff">': 1}, '<AUTHOR': {'name="Kenneth': 1, 'Rogoff">': 1, '<P>': 1}, 'name="Kenneth': {'Rogoff">': 1, '<P>': 1, 'SAN': 1}, 'Rogoff">': {'<P>': 1, 'SAN': 1, 'FRANCISCO': 1}, '<P>': {'SAN': 1, 'FRANCISCO': 1, '–': 1, 'Wouldn’t': 1, 'you': 2, 'know': 1, 'One': 2, 'successful': 1, 'gold': 1, 'Admittedly,': 1, 'getting': 1, 'to': 1, 'answer,': 1, 'of': 1, 'Sure,': 1, 'some': 1, 'might': 1, 'Even': 1, 'so,': 1, 'the': 2, 'There': 1, 'is': 1, 'probably': 1, 'In': 1, 'my': 1, 'view,': 1, 'At': 1, 'same': 1, 'So,': 1, 'yes,': 1, 'there': 1, 'Indeed,': 1, 'another': 1, 'critical': 1, 'Most': 1, 'economic': 1, 'research': 1, 'If': 1, 'are': 1, 'Of': 1, 'course,': 1, 'such': 1}, 'SAN': {'FRANCISCO': 1, '–': 1, 'It': 1}, 'FRANCISCO': {'–': 1, 'It': 1, 'has': 1}, '–': {'It': 1, 'has': 1, 'never': 1, 'and': 1, 'also': 1, 'a': 1, 'to': 1, 

In [42]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
#sentences = MySentences('/home/sebastien/Documents/Polytech/NLP/English/') # a memory-friendly iterator
def getDataForModel(PathToRead):
    logging.info("Generating the array of data in order to train the model with those data..")
    data = []
    onlyfiles = [f for f in listdir(PathToRead) if isfile(join(PathToRead, f))]
    for file in onlyfiles:
        with open(PathToRead+file, encoding ='utf8') as f:
            data.append(f.read().replace('\n', ' '))
    sentences = []
    sentencesCut = []
    for doc in data:
        sentences.append(doc.split(' . '))
    for sentence in sentences:
        for s in sentence:
            sTrim = re.sub(r',', ' ', s)
            sTrim = re.sub(r' ’ s', '', sTrim)
            sentencesCut.append(sTrim.split())
    logging.info("Array generated ! ")
    return (sentencesCut,sentences)

dataForModel,dataForFasttext = getDataForModel('D:/Documents/Polytech/NLP/leme/')

2019-11-24 11:30:48,002 : INFO : Generating the array of data in order to train the model with those data..
2019-11-24 11:30:51,697 : INFO : Array generated ! 


In [43]:
print(dataForModel[2])

['Just', 'December', 'fellow', 'economist', 'Martin', 'Feldstein', 'Nouriel', 'Roubini', 'penned', 'op-eds', 'bravely', 'questioning', 'bullish', 'market', 'sentiment', 'sensibly', 'pointing', 'gold', 'risk']


In [14]:
logging.info("Start to generate all the model, 2 skip gram (100/300) and 2 CBOW (100/300)")
models = []

logging.info("Generating skip-gram 300...")
model = gensim.models.Word2Vec(dataForModel,size=300,min_count=1,sg=1)
model.save("./model/word2vec300SG")
models.append((model,'Skip Gram 300'))
logging.info("Generating skip-gram 100...")
model = gensim.models.Word2Vec(dataForModel,size=100,min_count=1,sg=1)
model.save("./model/word2vec100SG")
models.append((model,'Skip Gram 100'))
logging.info("Generating CBOW 300...")
model = gensim.models.Word2Vec(dataForModel,size=300,min_count=1)
model.save("./model/word2vec300CBOW")
models.append((model,'CBOW 300'))
logging.info("Generating CBOW 100...")
model = gensim.models.Word2Vec(dataForModel,size=100,min_count=1)
model.save("./model/word2vec100CBOW")
models.append((model,'CBOW 100'))

logging.info("All models are now generated, time to test !")

2019-11-23 21:56:41,244 : INFO : Start to generate all the model, 2 skip gram (100/300) and 2 CBOW (100/300)
2019-11-23 21:56:41,328 : INFO : Generating skip-gram 300...
2019-11-23 21:56:41,328 : INFO : collecting all words and their counts
2019-11-23 21:56:41,329 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-11-23 21:56:41,355 : INFO : PROGRESS: at sentence #10000, processed 167000 words, keeping 17523 word types
2019-11-23 21:56:41,382 : INFO : PROGRESS: at sentence #20000, processed 337041 words, keeping 26301 word types
2019-11-23 21:56:41,412 : INFO : PROGRESS: at sentence #30000, processed 509923 words, keeping 32059 word types
2019-11-23 21:56:41,440 : INFO : PROGRESS: at sentence #40000, processed 675666 words, keeping 37429 word types
2019-11-23 21:56:41,470 : INFO : PROGRESS: at sentence #50000, processed 848467 words, keeping 42432 word types
2019-11-23 21:56:41,503 : INFO : PROGRESS: at sentence #60000, processed 1027013 words, keeping 4679

2019-11-23 21:57:06,157 : INFO : EPOCH 2 - PROGRESS: at 11.00% examples, 280373 words/s, in_qsize 5, out_qsize 0
2019-11-23 21:57:07,205 : INFO : EPOCH 2 - PROGRESS: at 16.46% examples, 281709 words/s, in_qsize 5, out_qsize 0
2019-11-23 21:57:08,264 : INFO : EPOCH 2 - PROGRESS: at 21.52% examples, 276457 words/s, in_qsize 5, out_qsize 0
2019-11-23 21:57:09,275 : INFO : EPOCH 2 - PROGRESS: at 26.73% examples, 275406 words/s, in_qsize 5, out_qsize 0
2019-11-23 21:57:10,321 : INFO : EPOCH 2 - PROGRESS: at 32.55% examples, 277308 words/s, in_qsize 5, out_qsize 0
2019-11-23 21:57:11,364 : INFO : EPOCH 2 - PROGRESS: at 38.29% examples, 278860 words/s, in_qsize 5, out_qsize 0
2019-11-23 21:57:12,411 : INFO : EPOCH 2 - PROGRESS: at 43.94% examples, 279661 words/s, in_qsize 5, out_qsize 0
2019-11-23 21:57:13,453 : INFO : EPOCH 2 - PROGRESS: at 49.67% examples, 280401 words/s, in_qsize 5, out_qsize 0
2019-11-23 21:57:14,519 : INFO : EPOCH 2 - PROGRESS: at 55.22% examples, 280592 words/s, in_qsiz

2019-11-23 21:58:11,770 : INFO : EPOCH 5 - PROGRESS: at 53.51% examples, 277166 words/s, in_qsize 5, out_qsize 0
2019-11-23 21:58:12,795 : INFO : EPOCH 5 - PROGRESS: at 58.75% examples, 277020 words/s, in_qsize 5, out_qsize 0
2019-11-23 21:58:13,801 : INFO : EPOCH 5 - PROGRESS: at 64.26% examples, 278123 words/s, in_qsize 5, out_qsize 0
2019-11-23 21:58:14,804 : INFO : EPOCH 5 - PROGRESS: at 69.04% examples, 276560 words/s, in_qsize 5, out_qsize 0
2019-11-23 21:58:15,841 : INFO : EPOCH 5 - PROGRESS: at 74.42% examples, 275739 words/s, in_qsize 5, out_qsize 0
2019-11-23 21:58:16,864 : INFO : EPOCH 5 - PROGRESS: at 80.34% examples, 276889 words/s, in_qsize 5, out_qsize 0
2019-11-23 21:58:17,868 : INFO : EPOCH 5 - PROGRESS: at 85.96% examples, 277130 words/s, in_qsize 5, out_qsize 0
2019-11-23 21:58:18,882 : INFO : EPOCH 5 - PROGRESS: at 91.73% examples, 277697 words/s, in_qsize 5, out_qsize 0
2019-11-23 21:58:19,897 : INFO : EPOCH 5 - PROGRESS: at 97.48% examples, 278635 words/s, in_qsiz

2019-11-23 21:58:33,018 : INFO : EPOCH 1 - PROGRESS: at 64.58% examples, 423385 words/s, in_qsize 5, out_qsize 0
2019-11-23 21:58:34,029 : INFO : EPOCH 1 - PROGRESS: at 73.11% examples, 425909 words/s, in_qsize 5, out_qsize 0
2019-11-23 21:58:35,035 : INFO : EPOCH 1 - PROGRESS: at 81.97% examples, 427889 words/s, in_qsize 5, out_qsize 0
2019-11-23 21:58:36,036 : INFO : EPOCH 1 - PROGRESS: at 90.72% examples, 428959 words/s, in_qsize 5, out_qsize 0
2019-11-23 21:58:37,069 : INFO : EPOCH 1 - PROGRESS: at 99.58% examples, 430074 words/s, in_qsize 4, out_qsize 0
2019-11-23 21:58:37,073 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-11-23 21:58:37,086 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-11-23 21:58:37,120 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-11-23 21:58:37,121 : INFO : EPOCH - 1 : training on 6297818 raw words (5245932 effective words) took 12.2s, 430300 effective words/s
2019-11-23 21:58:38,134 

2019-11-23 21:59:25,850 : INFO : storing np array 'syn1neg' to ./model/word2vec100SG.trainables.syn1neg.npy
2019-11-23 21:59:25,999 : INFO : not storing attribute cum_table
2019-11-23 21:59:26,211 : INFO : saved ./model/word2vec100SG
2019-11-23 21:59:26,212 : INFO : Generating CBOW 300...
2019-11-23 21:59:26,212 : INFO : collecting all words and their counts
2019-11-23 21:59:26,212 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-11-23 21:59:26,237 : INFO : PROGRESS: at sentence #10000, processed 167000 words, keeping 17523 word types
2019-11-23 21:59:26,263 : INFO : PROGRESS: at sentence #20000, processed 337041 words, keeping 26301 word types
2019-11-23 21:59:26,290 : INFO : PROGRESS: at sentence #30000, processed 509923 words, keeping 32059 word types
2019-11-23 21:59:26,316 : INFO : PROGRESS: at sentence #40000, processed 675666 words, keeping 37429 word types
2019-11-23 21:59:26,343 : INFO : PROGRESS: at sentence #50000, processed 848467 words, keepi

2019-11-23 21:59:46,886 : INFO : EPOCH 3 - PROGRESS: at 85.12% examples, 890079 words/s, in_qsize 5, out_qsize 0
2019-11-23 21:59:47,690 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-11-23 21:59:47,693 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-11-23 21:59:47,704 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-11-23 21:59:47,705 : INFO : EPOCH - 3 : training on 6297818 raw words (5246991 effective words) took 5.9s, 896093 effective words/s
2019-11-23 21:59:48,710 : INFO : EPOCH 4 - PROGRESS: at 16.61% examples, 883891 words/s, in_qsize 6, out_qsize 1
2019-11-23 21:59:49,716 : INFO : EPOCH 4 - PROGRESS: at 33.51% examples, 884987 words/s, in_qsize 5, out_qsize 0
2019-11-23 21:59:50,727 : INFO : EPOCH 4 - PROGRESS: at 51.22% examples, 897227 words/s, in_qsize 6, out_qsize 0
2019-11-23 21:59:51,731 : INFO : EPOCH 4 - PROGRESS: at 68.75% examples, 907519 words/s, in_qsize 5, out_qsize 0
2019-11-23 21:59:52,736 :

2019-11-23 22:00:02,541 : INFO : estimated required memory for 115867 words and 100 dimensions: 150627100 bytes
2019-11-23 22:00:02,541 : INFO : resetting layer weights
2019-11-23 22:00:03,669 : INFO : training model with 3 workers on 115867 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
2019-11-23 22:00:04,677 : INFO : EPOCH 1 - PROGRESS: at 24.45% examples, 1308075 words/s, in_qsize 5, out_qsize 0
2019-11-23 22:00:05,683 : INFO : EPOCH 1 - PROGRESS: at 50.11% examples, 1320037 words/s, in_qsize 5, out_qsize 0
2019-11-23 22:00:06,685 : INFO : EPOCH 1 - PROGRESS: at 75.58% examples, 1329154 words/s, in_qsize 5, out_qsize 0
2019-11-23 22:00:07,665 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-11-23 22:00:07,674 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-11-23 22:00:07,678 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-11-23 22:00:07,679 : INFO : EPOCH - 1 : training on 6297818 

In [6]:
#mymodel = gensim.models.Word2Vec.load('./model/word2vec200cbow')


2019-11-23 21:36:54,592 : INFO : loading Word2Vec object from ./model/word2vec200cbow
2019-11-23 21:36:54,992 : INFO : loading wv recursively from ./model/word2vec200cbow.wv.* with mmap=None
2019-11-23 21:36:54,993 : INFO : loading vectors from ./model/word2vec200cbow.wv.vectors.npy with mmap=None
2019-11-23 21:36:55,574 : INFO : setting ignored attribute vectors_norm to None
2019-11-23 21:36:55,575 : INFO : loading vocabulary recursively from ./model/word2vec200cbow.vocabulary.* with mmap=None
2019-11-23 21:36:55,575 : INFO : loading trainables recursively from ./model/word2vec200cbow.trainables.* with mmap=None
2019-11-23 21:36:55,576 : INFO : loading syn1neg from ./model/word2vec200cbow.trainables.syn1neg.npy with mmap=None
2019-11-23 21:36:56,083 : INFO : setting ignored attribute cum_table to None
2019-11-23 21:36:56,084 : INFO : loaded ./model/word2vec200cbow


In [12]:
#ten words to test : Euro,inanimate,Cyprus,China,dna,music,yahoo,wrong,mouth,singer,rich
# un PDF + un readme

wordList = ['Euro','inanimate','Cyprus','China','music','Yahoo','wrong','mouth','singer','rich']
def selection_sort(x,word):
    for i in range(len(x)):
        swap = i + np.argmax(x[i:])
        (x[i], x[swap]) = (x[swap], x[i])
        (word[i], word[swap]) = (word[swap], word[i])
    return [x,word]

def getSimilarity(myWord,mymodel):
    arrayWord = []
    arraySimilarities = []
    i = 0
    for word in mymodel.wv.vocab:
        cosineSim = mymodel.wv.similarity(myWord,word)
        if(cosineSim > 0.2 and cosineSim != 1):
            arraySimilarities.append((mymodel.wv.similarity(myWord,word),word))
    arraySimilarities.sort(key=lambda tup: tup[0],reverse=True)
    print(arraySimilarities[:10])
    print("\n")
for model in models:
    logging.info("-----------------Testing word similarity for "+model[1]+"-----------------")
    for word in wordList:
        print('Word used :' + word)
        getSimilarity(word,model[0])
#getSimilarity('Britany')
#print(mymodel.wv.similarity('Euro','Cyprus'))
#matrix_similarity = cosine_similarity(mymodel.wv)

NameError: name 'models' is not defined

In [44]:
import treetaggerwrapper
def removeStopWords(pathToStopWords,pathToRead,pathToWrite):
    stopWords = []
    with open(pathToStopWords,encoding='utf8') as f:
        for line in f:
            stopWords.append(line[:-1])
    onlyfiles = [f for f in listdir(pathToRead) if isfile(join(pathToRead, f))]
    for file in onlyfiles:
        with open(pathToRead + file, "r",encoding='utf8') as f:
            fwrite = open(pathToWrite + file + ".txt", "w",encoding='utf8')
            for line in f:
                for word in line.split():
                    if word not in stopWords:
                        fwrite.write(word+' ')
                fwrite.write('\n')
#removeStopWords('D:/Documents/Polytech/NLP/stopword-en.txt','D:/Documents/Polytech/NLP/English/','D:/Documents/Polytech/NLP/filteredTxt/')

def tokenisation(pathToRead, pathToWrite):
    onlyfiles = [f for f in listdir(pathToRead) if isfile(join(pathToRead, f))]
    for file in onlyfiles:
        with open(pathToRead + file, "r",encoding='utf8') as f:
            fwrite = open(pathToWrite + file + ".tok", "w",encoding='utf8')
            for line in f:
                fwrite.write(' '.join(nltk.word_tokenize(line)))
                fwrite.write('\n')
#tokenisation('D:/Documents/Polytech/NLP/filteredTxt/','D:/Documents/Polytech/NLP/tok/')
def lemmatisation(pathToRead,pathToWrite):
    onlyfiles = [f for f in listdir(pathToRead) if isfile(join(pathToRead, f))]
    i = 1
    for file in onlyfiles:
        with open(pathToRead + file, "r",encoding='utf8') as f:
            fwrite = open(pathToWrite + os.path.basename(f.name) + ".lem", "w",encoding='utf8')
            lemmatizer = WordNetLemmatizer()
            for line in f:
                for word in line.split():
                    fwrite.write(lemmatizer.lemmatize(word) + " ")
                fwrite.write('\n')

#lemmatisation('D:/Documents/Polytech/NLP/tok/','D:/Documents/Polytech/NLP/leme/')
def treeTagger(pathToRead, pathToWrite):
    onlyfiles = [f for f in listdir(pathToRead) if isfile(join(pathToRead, f))]
    i = 1
    for file in onlyfiles:
        fwrite = pathToWrite + file + ".lem"
        os.system("python D:/IDE/Anaconda/Lib/site-packages/treetaggerwrapper.py -d C:/TreeTagger/ --pipe < " +
                  pathToRead + file + " > " + fwrite)
        print(i)
        i += 1
#treeTagger('D:/Documents/Polytech/NLP/leme/','D:/Documents/Polytech/NLP/tag/')
tagger = treetaggerwrapper.TreeTagger(TAGLANG='en')

print(len(dataForFasttext))
#tags = tagger.tag_text(dataForFasttext)
#lemmatisation('/home/sebastien/Documents/Polytech/NLP/token/','/home/sebastien/Documents/Polytech/NLP/realLem/')

2019-11-24 11:31:29,759 : INFO : lang=en
2019-11-24 11:31:29,759 : INFO : Use previously found TreeTagger directory: C:\TreeTagger
2019-11-24 11:31:29,760 : INFO : tagdir=C:\TreeTagger
2019-11-24 11:31:29,761 : INFO : tagbin=C:\TreeTagger\bin\tree-tagger.exe
2019-11-24 11:31:29,762 : INFO : tagparfile=C:\TreeTagger\lib\english.par
2019-11-24 11:31:29,763 : INFO : tagopt=-token -lemma -sgml -quiet -no-unknown
2019-11-24 11:31:29,763 : INFO : taginencoding=utf-8
2019-11-24 11:31:29,764 : INFO : tagoutencoding=utf-8
2019-11-24 11:31:29,765 : INFO : taginencerr=replace
2019-11-24 11:31:29,765 : INFO : tagoutencerr=replace
2019-11-24 11:31:29,766 : INFO : abbrevfile=C:\TreeTagger\lib\english-abbreviations
2019-11-24 11:31:29,767 : INFO : Read 289 abbreviations from file: C:\TreeTagger\lib\english-abbreviations


10056


In [25]:
from gensim.test.utils import datapath

for model in models:
    logging.info("-----------------Testing word pairs with wordsim353 for "+model[1]+"-----------------")
    similarities = model[0].wv.evaluate_word_pairs(datapath('wordsim353.tsv'))
    print('\n')
    print("Similarities resume for "+ model[1])
    print(similarities)

2019-11-23 22:07:33,252 : INFO : -----------------Testing word pairs with wordsim353 for Skip Gram 300-----------------
2019-11-23 22:07:33,356 : INFO : Pearson correlation coefficient against D:\IDE\Anaconda\lib\site-packages\gensim\test\test_data\wordsim353.tsv: 0.4465
2019-11-23 22:07:33,357 : INFO : Spearman rank-order correlation coefficient against D:\IDE\Anaconda\lib\site-packages\gensim\test\test_data\wordsim353.tsv: 0.4600
2019-11-23 22:07:33,357 : INFO : Pairs with unknown words ratio: 2.3%
2019-11-23 22:07:33,362 : INFO : -----------------Testing word pairs with wordsim353 for Skip Gram 100-----------------




Similarities resume for Skip Gram 300
((0.4465322056088227, 2.6050543509703988e-18), SpearmanrResult(correlation=0.45999281980500845, pvalue=1.8168672991229562e-19), 2.26628895184136)


2019-11-23 22:07:33,715 : INFO : Pearson correlation coefficient against D:\IDE\Anaconda\lib\site-packages\gensim\test\test_data\wordsim353.tsv: 0.4573
2019-11-23 22:07:33,716 : INFO : Spearman rank-order correlation coefficient against D:\IDE\Anaconda\lib\site-packages\gensim\test\test_data\wordsim353.tsv: 0.4388
2019-11-23 22:07:33,717 : INFO : Pairs with unknown words ratio: 2.3%
2019-11-23 22:07:33,722 : INFO : -----------------Testing word pairs with wordsim353 for CBOW 300-----------------
2019-11-23 22:07:33,831 : INFO : Pearson correlation coefficient against D:\IDE\Anaconda\lib\site-packages\gensim\test\test_data\wordsim353.tsv: 0.4427
2019-11-23 22:07:33,832 : INFO : Spearman rank-order correlation coefficient against D:\IDE\Anaconda\lib\site-packages\gensim\test\test_data\wordsim353.tsv: 0.4330
2019-11-23 22:07:33,832 : INFO : Pairs with unknown words ratio: 2.3%
2019-11-23 22:07:33,836 : INFO : -----------------Testing word pairs with wordsim353 for CBOW 100----------------



Similarities resume for Skip Gram 100
((0.4573040351610507, 3.1220745477851015e-19), SpearmanrResult(correlation=0.43880417186724185, pvalue=1.1402298964459586e-17), 2.26628895184136)


Similarities resume for CBOW 300
((0.4426804743688478, 5.462951542709803e-18), SpearmanrResult(correlation=0.4330235767285205, pvalue=3.3574951916403924e-17), 2.26628895184136)


2019-11-23 22:07:33,941 : INFO : Pearson correlation coefficient against D:\IDE\Anaconda\lib\site-packages\gensim\test\test_data\wordsim353.tsv: 0.4417
2019-11-23 22:07:33,942 : INFO : Spearman rank-order correlation coefficient against D:\IDE\Anaconda\lib\site-packages\gensim\test\test_data\wordsim353.tsv: 0.4359
2019-11-23 22:07:33,942 : INFO : Pairs with unknown words ratio: 2.3%




Similarities resume for CBOW 100
((0.44166686869988636, 6.6280326569271e-18), SpearmanrResult(correlation=0.4358708423909942, pvalue=1.9774930782925625e-17), 2.26628895184136)


In [37]:
#import fasttext
#model = fasttext.train_unsupervised('data/fil9')

def makeAllLemmeTogether(pathToLemme):
    onlyfiles = [f for f in listdir(pathToLemme) if isfile(join(pathToLemme, f))]
    fwrite = open(pathToLemme + "all.lem", "w",encoding='utf8')
    i = 1
    for file in onlyfiles:
        print(i)
        i +=1
        with open(pathToLemme + file, "r",encoding='utf8') as f:
            for line in f:
                fwrite.write(line)
#makeAllLemmeTogether("D:/Documents/Polytech/NLP/leme/")

fwrite = open('D:/Documents/Polytech/NLP/fasttext/' + "corpus.txt", "w",encoding='utf8')
for doc in dataForFasttext:
    for sentence in doc:
        fwrite.write('1,"'+sentence+'"')

44


In [2]:
import fasttext

logging.info("-----------------Generating SkipGram model with FastText-----------------")
model = fasttext.train_unsupervised("D:/Documents/Polytech/NLP/fasttext/corpus.txt", model='skipgram')
model.save_model("fastTextSkipGram.bin")
logging.info("-----------------Generating CBOW model with FastText-----------------")
model = fasttext.train_unsupervised("D:/Documents/Polytech/NLP/fasttext/corpus.txt", model='cbow')
model.save_model("fastTextCBOW.bin")


In [10]:
from gensim.models.wrappers import FastText
fastModels = []
model = FastText.load_fasttext_format('fastTextCBOW.bin')
fastModels.append((model,'FastTextCBOW'))

model = FastText.load_fasttext_format('fastTextSkipGram.bin')
fastModels.append((model,'FastTextSkipGram'))

In [13]:
for model in fastModels:
    logging.info("-----------------Testing word similarity for "+model[1]+"-----------------")
    for word in wordList:
        print('Word used :' + word)
        getSimilarity(word,model[0])

Word used :Euro
[(0.88166994, 'EurAsEC'), (0.86298823, 'Euro-zone'), (0.8236492, 'euro'), (0.8172005, 'Eurozone'), (0.8057518, 'Europol'), (0.80007076, 'Europe.'), (0.79972875, 'Europes'), (0.7994734, 'EMU'), (0.7878887, 'Europhiles'), (0.78172755, 'Eurostat')]


Word used :inanimate
[(0.8888814, 'animate'), (0.8431741, 'intimate'), (0.84179956, 'inmate'), (0.8104749, 'decimate'), (0.7888785, 'glutamate'), (0.786278, 'primate'), (0.7844594, 'climate'), (0.7625569, 'mate'), (0.75348246, 'climate-related'), (0.7179187, 'ultimate')]


Word used :Cyprus
[(0.8428812, 'Cypriots'), (0.83738166, 'Cypriot'), (0.701944, 'Greek-Turkish'), (0.65842605, 'Catalonia'), (0.64819086, 'Ireland'), (0.6476697, 'Greece'), (0.64051557, 'Greek'), (0.63694596, 'Slovenia'), (0.6318108, 'Balkans'), (0.6303593, 'कितनी')]


Word used :China
[(0.96617216, 'China.'), (0.947366, 'China-US'), (0.94100684, 'Chinas'), (0.9395753, 'China-EU'), (0.88856035, 'India-China'), (0.88564396, 'US-China'), (0.8058939, 'pro-China

In [14]:
for model in fastModels:
    logging.info("-----------------Testing word pairs with wordsim353 for "+model[1]+"-----------------")
    similarities = model[0].wv.evaluate_word_pairs(datapath('wordsim353.tsv'))
    print('\n')
    print("Similarities resume for "+ model[1])
    print(similarities)



Similarities resume for FastTextCBOW
((0.4289160941819007, 1.4216866666658386e-15), SpearmanrResult(correlation=0.404601579015805, pvalue=7.07934279965324e-14), 10.48158640226629)


Similarities resume for FastTextSkipGram
((0.5419461065390799, 1.6095150633283194e-25), SpearmanrResult(correlation=0.5429516705203242, pvalue=1.2604008241500752e-25), 10.48158640226629)
